In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
!pip install tokenizers
from tokenizers.normalizers import NFD


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [26]:
## TORCH IMPORTS
import torch
from torch.nn.utils.rnn import pad_sequence
from torch import nn, Tensor,multinomial
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer, Transformer
from torch.utils.data import dataset, DataLoader
from torch.autograd import Variable
from torchtext.vocab import Vectors, vocab,build_vocab_from_iterator,vocab

## OTHER IMPORTS
import math
import re
import pickle
from collections import defaultdict, OrderedDict, Counter
from tqdm import tqdm
import numpy as np
from numpy.random import uniform

In [27]:
## CONSTANTS
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
langs_codes = {"Gitksan": "git", "Arapaho":"arp", "Lezgi":"lez", "Nyangbo":"nyb", "Tsez":"ddo","Uspanteko":"usp"}
START = "<start>"
END = "<end>"
UNK = "<unk>"
PAD = "<pad>"
BD = "<bd>"
SP = "#"
SPECIALS = [PAD, START, END,UNK,SP]
normalizer = NFD()
BATCH_SIZE = 16

In [35]:


def get_segmented_data(ld,splits=["transcriptions","glosses"]):
    def get_segmented(line):
        allsplits=[]

        splits=[m.split() for m in line]
        for i in splits:
            for m in i:
                allsplits.append(m)
        return allsplits

    data = {}
    for split in splits:
        data[split] = get_segmented(ld[split])
    return data
    
def tag_bd(seq):

  for i, c in enumerate(seq):
    if i < len(seq)-2:
      if seq[i+1] == "-":
          seq[i] += seq[i+1]
          seq[i+2] = seq[i+1] + seq[i+2]
  return [i for i in seq if not i =="-"]

orth_process = lambda x: [START] + list(re.sub(" ", "#", x)) + [END]
gloss_process = lambda x: [START] + tag_bd(list(re.sub(" ", "#", x))) + [END]


def get_linesplits(orths,glosses):
  ex=[]
  for o,g in zip(orths, glosses):
    orth = orth_process(o)
    gloss = gloss_process(g)
    ex.append((orth,gloss))
   
  return ex


def get_train_val_split(data, props=10):
    test_data = []
    train_data = []
    for i, dat in enumerate(data):
        if i%props == 0:
            test_data.append(dat)
        else:
            train_data.append(dat)
    return test_data, train_data


def get_windows(glosses,prop=0.5):
    counts=defaultdict(int)
    for gloss in glosses:
        counts[len(gloss.split())]+=1
    
    sorted_counts = sorted(counts.items(),key=lambda x:x[1],reverse=True)
    # get upper limit by finding average, then the max len of the lower prop% below average
    lcounts = [l for l,c in sorted_counts]
    avg = round(sum(lcounts)/len(lcounts))*prop
    
    while True:
        windows=np.random.randint(1,avg,int(avg-2))
        if len(set(windows)) == int(avg-2):
            break
            
    return windows

def get_window_segs(transcriptions,glosses, windows):
    
    window_segs = []
    for trans,gls in zip(transcriptions,glosses):
        transsplit=trans.split()
        glssplit=gls.split()
        for window in windows:
            if len(glssplit) < window:
                continue
            sid = 0
            for i in range(len(glssplit)-1):
                transwind = transsplit[sid:sid+window]
                glsswind = glssplit[sid:sid+window]
                
                if len(transwind) < window:
                    continue
                window_segs.append((orth_process(" ".join(transwind)),gloss_process(" ".join(glsswind))))

                sid+=1
    return window_segs
    

class LangIGT:
    def __init__(self, lang):
      
        self.lang = lang
        self.batch_size = BATCH_SIZE
        
        self.train_ld, self.dev_ld = self.get_langdict()
        self.windows = get_windows(self.train_ld["glosses"])

        self.train_splits, self.dev_splits = get_linesplits(self.train_ld["transcriptions"],self.train_ld["glosses"]), get_linesplits(self.dev_ld["transcriptions"],self.dev_ld["glosses"])
        self.train_windows, self.dev_windows = get_window_segs(self.train_ld["transcriptions"],self.train_ld["glosses"], self.windows), get_window_segs(self.dev_ld["transcriptions"],self.dev_ld["glosses"],self.windows)
        self.gls_alphabet, self.orth_alphabet = self.get_alphabet()
        
    
    def get_langdict(self):
        ld = {
            "train": defaultdict(list),
            "dev": defaultdict(list)
        }
        path = f"/content/drive/MyDrive/2023glossingST/data/{self.lang}/"
        #path = f"data/{self.lang}/"
        train_fn = f"{langs_codes[self.lang]}-train-track2-uncovered"
        dev_fn = f"{langs_codes[self.lang]}-dev-track2-uncovered"

        for fp in [train_fn, dev_fn]:
            data_type = fp.split("-")[1]
            with open(path + fp, "r",encoding="utf-8") as f:
                for line in f.readlines():

                    if line.startswith("\\t"):
                        ld[data_type]["transcriptions"].append(line.lstrip("\\t ").rstrip("\n"))
                    if line.startswith("\g"):
                        ld[data_type]["glosses"].append(line.lstrip("\\g ").rstrip("\n"))
                    if line.startswith("\l"):
                        ld[data_type]["translation"].append(line.lstrip("\\l ").rstrip("\n"))

        return ld["train"],ld["dev"]


    def get_alphabet(self):
      gloss_counter, orth_counter = Counter(), Counter()
      for gloss,trans in zip(self.train_ld["glosses"],self.train_ld["transcriptions"]):
        for g,t in zip(tag_bd(list(re.sub(" ", "#", gloss))), list(re.sub(" ", "#", trans))):
          gloss_counter.update([g])
          orth_counter.update([t])

      gloss_voc = vocab(OrderedDict(sorted(gloss_counter.items(), key=lambda x: x[1], reverse=True)), specials=SPECIALS)
      gloss_voc.set_default_index(gloss_voc[UNK])
      orth_voc = vocab(OrderedDict(sorted(orth_counter.items(), key=lambda x: x[1], reverse=True)), specials=SPECIALS)
      orth_voc.set_default_index(orth_voc[UNK])
      return gloss_voc, orth_voc

    def data(self,split):
      train, test = [], []

      for i in self.train_splits:
        train.append(i)
      for j in self.train_windows:
        train.append(j)

      for k in self.dev_splits:
        test.append(k)
      for l in self.dev_windows:
        test.append(l)

      dev, train = get_train_val_split(train)
      
      

      data = {
            "train" : train,
            "dev" : dev,
            "test" : test
        }
      return data[split]

    def alphabet(self,split):
      alphabet = {
            "gloss" : self.gls_alphabet,
            "orth" : self.orth_alphabet
        }
      return alphabet[split]





In [36]:
lang = LangIGT("Lezgi")
train,dev, test = lang.data("train"), lang.data("dev"), lang.data("test")
gloss_alpha, orth_alpha = lang.alphabet("gloss"), lang.alphabet("orth")

In [37]:
print(f"window sizes: {lang.windows}")
print(f"size of training set: {len(train)}")
print(f"size of dev set: {len(dev)}")
print(f"size of test set: {len(test)}")

window sizes: [8 6 7 4 5 3 1]
size of training set: 28308
size of dev set: 3146
size of test set: 4705


In [47]:
def tensor_transform(data):
  orths=[]
  for orth, _ in data:
    input_tensor = torch.tensor([orth_alpha[i] for i in orth], dtype=torch.long)
    orths.append(input_tensor)
  return orths
  
def collate_batch(batch):
    
    input, output=[], []
    for morphemes, glosses in batch:
        input_tensor = torch.tensor([orth_alpha[i] for i in morphemes], dtype=torch.long)
        output_tensor = torch.tensor([gloss_alpha[i] for i in glosses], dtype=torch.long)
        input.append(input_tensor)
        output.append(output_tensor)
        # input_lengths.append(input_tensor.shape[0])
        # output_lengths.append(output_tensor.shape[0])


    return pad_sequence(input, 
                        batch_first=False, 
                        padding_value=orth_alpha["<pad>"]), pad_sequence(output, 
                        batch_first=False, 
                        padding_value=gloss_alpha["<pad>"])

# Example((pad_sequence(input, 
#                                  batch_first=False, 
#                                  padding_value=src_alphabet["<pad>"]), torch.tensor(input_lengths, dtype=torch.long)),
#                        (pad_sequence(output, 
#                                  batch_first=False, 
#                                  padding_value=trg_alphabet["<pad>"]), torch.tensor(output_lengths, dtype=torch.long)))


def torchify(train,dev):
    path = "/content/drive/MyDrive/2023glossingST/"
    
    train_dataloader = DataLoader(
        train,
        batch_size=BATCH_SIZE,
        collate_fn=collate_batch,
        shuffle=True
    )

    dev_dataloader = DataLoader(
        dev,
        batch_size=BATCH_SIZE,
        collate_fn=collate_batch,
        shuffle=False
    )

    test_dataloader = DataLoader(
        test,
        batch_size=BATCH_SIZE,
        collate_fn=collate_batch,
        shuffle=False
    )
    return train_dataloader, dev_dataloader, test_dataloader

    

In [48]:
train_dataloader, dev_dataloader, test_dataloader = torchify(train,dev)

In [50]:
for batch in train_dataloader:
    src, trg = batch
    print('tensor size of source language:', src.shape)
    print('tensor size of target language:', trg.shape)
    print('the tensor of first example in target language:', trg[:, 0])
    print('the tensor of first example in src language:', src[:, 0])
    print([orth_alpha.get_itos()[i] for i in src[:, 0]])
    print([gloss_alpha.get_itos()[i] for i in trg[:, 0]])
    break

tensor size of source language: torch.Size([54, 16])
tensor size of target language: torch.Size([78, 16])
the tensor of first example in target language: tensor([ 1, 37,  8, 17, 12,  6, 26, 53, 23, 21, 68,  4, 51, 28, 14,  6, 13,  4,
        17, 30, 13, 53, 52, 16, 39,  4,  7,  6, 45, 32, 23, 16, 39,  2,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0])
the tensor of first example in src language: tensor([ 1, 17,  8,  7,  7,  6,  4, 38, 24, 12,  5,  7,  4, 20, 34, 13,  7, 14,
         7,  6,  4, 16, 22,  5, 25,  8,  7,  5,  2,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
['<start>', 'з', 'у', 'н', 'н', 'и', '#', 'И', 'м', 'р', 'а', 'н', '#', 'т', 'ф', 'е', 'н', 'г', 'н', 'и', '#', 'к', 'ъ', 'а', 'ч', 'у', 'н', 'а', '<end>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>

In [51]:

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = max(len(i) for i in [g for _,g in train])+5):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, emb_size, embedding: Vectors):
        super(TokenEmbedding, self).__init__()
        #self.embedding = nn.Embedding(vocab_size, emb_size)
        self.embed = embedding
        #self.emb_size = emb_size
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embed(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 src_embeddings: nn.Embedding,
                 tgt_embeddings: Vectors,
                 dim_feedforward: int = 512,
                 dropout: float = 0.2):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(emb_size, src_embeddings)
        self.tgt_tok_emb = TokenEmbedding(emb_size, tgt_embeddings)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [52]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == orth_alpha.get_stoi()[PAD]).transpose(0, 1)
    tgt_padding_mask = (tgt == gloss_alpha.get_stoi()[PAD]).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [64]:

torch.manual_seed(531)

SRC_VOCAB_SIZE = len(orth_alpha)
TGT_VOCAB_SIZE = len(gloss_alpha)
EMB_SIZE = 300  # GloVe dim
NHEAD = 6
FFN_HID_DIM = 512
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

src_embeddings = nn.Embedding(SRC_VOCAB_SIZE, EMB_SIZE)
tgt_embeddings = nn.Embedding(TGT_VOCAB_SIZE, EMB_SIZE)


transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, src_embeddings,
                                 tgt_embeddings, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=0)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [65]:

def train_epoch(model, iterator, optimizer):
    model.train()
    losses = 0

    for src, tgt in tqdm(iterator):
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()

    return losses / len(iterator)


def evaluate(model, iterator):
    model.eval()
    losses = 0

    for src, tgt in iterator:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(iterator)

In [66]:
from timeit import default_timer as timer

def train_eval(NUM_EPOCHS, transformer, train_dataloader, optimizer, dev_dataloader, lang_code):
    for epoch in range(1, NUM_EPOCHS + 1):
        start_time = timer()
        train_loss = train_epoch(transformer, train_dataloader, optimizer)
        end_time = timer()
        val_loss = evaluate(transformer, dev_dataloader)

        state_dict_model = transformer.state_dict() 

        if epoch % 2 == 0:

          torch.save({
                'epoch': epoch,
                'model_state_dict': transformer.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_loss': val_loss,

                }, "/content/drive/MyDrive/2023glossingST/ckpt" + lang_code + "-" + str(epoch))
          print(f"Checkpoint saved at epoch={epoch}")

        print((f"Epoch: {epoch},  "f"Epoch time = {(end_time - start_time):.3f}s"))
        print(f'\t Train Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
        print(f'\t Val. Loss: {val_loss:.3f} |  Val. PPL: {math.exp(val_loss):7.3f}')

    return transformer

In [ ]:
train_eval(40, transformer, train_dataloader, optimizer, dev_dataloader, "lz")

100%|██████████| 1770/1770 [00:47<00:00, 37.09it/s]


Epoch: 1,  Epoch time = 47.730s
	 Train Loss: 2.165 | Train PPL:   8.717
	 Val. Loss: 1.456 |  Val. PPL:   4.288


100%|██████████| 1770/1770 [00:48<00:00, 36.33it/s]


Checkpoint saved at epoch=2
Epoch: 2,  Epoch time = 48.730s
	 Train Loss: 1.498 | Train PPL:   4.474
	 Val. Loss: 1.034 |  Val. PPL:   2.811


100%|██████████| 1770/1770 [00:48<00:00, 36.46it/s]


...

100%|██████████| 1770/1770 [00:49<00:00, 36.12it/s]


Epoch: 21,  Epoch time = 49.018s
	 Train Loss: 0.186 | Train PPL:   1.204
	 Val. Loss: 0.039 |  Val. PPL:   1.040


100%|██████████| 1770/1770 [00:48<00:00, 36.77it/s]


Checkpoint saved at epoch=22
Epoch: 22,  Epoch time = 48.150s
	 Train Loss: 0.174 | Train PPL:   1.190
	 Val. Loss: 0.036 |  Val. PPL:   1.037


  2%|▏         | 31/1770 [00:01<01:01, 28.11it/s]

In [ ]:
transformer_best = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, src_embeddings,
                                 tgt_embeddings, FFN_HID_DIM)

transformer_best.load_state_dict(torch.load('/content/drive/MyDrive/2023glossingST/ckpt/tz80.pt')['state_dict'])
transformer_best = transformer_best.to(DEVICE)

In [ ]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == gloss_alpha.get_stoi()[END]:
            break
    return ys

# actual function to translate input sentence into target language
def translate(model: torch.nn.Module, src_tensor, extend_by=20):
    model.eval()
    src = src_tensor.view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  
        src, 
        src_mask, 
        max_len=num_tokens + extend_by, 
        start_symbol=orth_alpha.get_stoi()[START]
    ).flatten()

    return re.sub("<start>|<end>", "", ''.join([gloss_alpha.get_itos()[t] for t in tgt_tokens]))#" ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

In [ ]:
t=tensor_transform(lang.dev_splits)
preds = [translate(transformer_best,o) for o in t]
preds=[p.replace("--","-").replace("@", " ??? ").replace("#", " ") for p in preds]

In [ ]:
with open("/content/drive/MyDrive/2023glossingST/predictions/" + "tsez", "w", encoding = "utf-8") as fout:
      fout.write("\n".join(preds))


In [ ]:
langdict=lang.dev_ld

In [ ]:
pred_lines=[]
for t, g,l in zip(langdict["transcriptions"], preds, langdict["translation"]):
    pred_lines.append("\\t " + t)
    pred_lines.append("\g " + g)
    pred_lines.append("\l " + l)
    pred_lines.append(" ")

In [ ]:
pred_lines

['\\t « Зун », лагьана , « фена инсанрин арада гьатда , акван белке зи кьисметда ава .»',
 '\\g « And.but that , boy-AOR but , bat bathat be-ENT , « hat be-ENT hat be-ERG bring-DIR-GEN brie , bring-DAT bring-DIR-GEN bring-AOR , theathe.bris brin-IR-GENEN , say-AOR .',
 '\\l "I will enter amongst the people.  Let me look - maybe it is my fate."',
 ' ',
 '\\t икьрар сад я .',
 '\\g boy-DIR-GEN brithat be-ENT boy-ERG , brithat be-ENT be-INESS , brithis be-INES be-INESS .',
 '\\l the decision is one',
 ' ',
 '\\t « Бес чун гьабурун патав хъфена кӀанда , Ша кӀватӀ хьана хъфин чун .»',
 '\\g « And.but that bot be-ENT that be-ENT bat , bat be-ENT hat be-ENT be-AOR , bring-D bring-DIR-GEN that bring-DAT bring-D , bring-D bring-DIR-GEN be-AT .',
 '\\l "But we need to return to them. Come, let\'s go back as a group."',
 ' ',
 '\\t Ашкъи вуш я , ашукь гада ?',
 '\\g boy-ERG that be-ENT this boy-ENT , brithat be-ENT be-INESS-SBS , brititis be-ENT be-INESS-S , brithis be-INES be-INEG .',
 '\\l What